In [1]:
try:
    import graphene
except ImportError:
    !/opt/conda/envs/python2/bin/pip install graphene

In [2]:
from collections import namedtuple
from itertools import groupby
from operator import itemgetter
import json
import omero.clients
from omero.rtypes import unwrap, rlist, rstring
from idr import connection
import graphene

c = connection()
qs = c.getQueryService()

Connected to IDR ...


In [3]:
class AnnotationType(graphene.Enum):
    GENE = 'gene'
    PHENOTYPE = 'phenotype'


class NameValue(graphene.ObjectType):
    name = graphene.String(required=True)
    value = graphene.String(required=True)


class Annotation(graphene.Interface):
    id = graphene.Int(required=True)
    name = graphene.String(required=True)
    images = graphene.List(graphene.NonNull(lambda: Image))
    value = graphene.List(graphene.NonNull(lambda: NameValue))


class AbstractAnnotation(graphene.AbstractType):
    id = graphene.Int(required=True)
    name = graphene.String(required=True)
    images = graphene.List(graphene.NonNull(lambda: Image))
    value = graphene.List(graphene.NonNull(lambda: NameValue))

    def resolve_images(self, info):
        p = omero.sys.ParametersI()
        p.addId(self.id)
        rs = qs.projection(
            """
            SELECT ial.parent.id, ial.parent.name
            FROM ImageAnnotationLink ial
            WHERE ial.child.id=:id
            AND ial.child.class=MapAnnotation
            """, p)
        return (Image(*r) for r in unwrap(rs))

class Gene(graphene.ObjectType, AbstractAnnotation):
    class Meta:
        interfaces = (Annotation,)
    ensemblid = graphene.String(required=True)

    #qs.projection("SELECT ann.id,ann.class,mv.name,mv.value,ann.ns FROM ImageAnnotationLink ial JOIN ial.child as ann JOIN ann.mapValue as mv WHERE parent=1030631 AND ann.class=MapAnnotation",None) 
    #def resolve_images



class Phenotype(graphene.ObjectType, AbstractAnnotation):
    class Meta:
        interfaces = (Annotation,)
    cmpoterm = graphene.String(required=True)


class Image(graphene.ObjectType):
    id = graphene.Int(required=True)
    name = graphene.String(required=True)
    annotations = graphene.List(
        graphene.NonNull(lambda: Annotation), mapr=AnnotationType())

    def resolve_annotations(self, info, mapr=None):
        p = omero.sys.ParametersI()
        p.addId(self.id)
        nsmap = NSMAP
        if mapr:
            k = 'openmicroscopy.org/mapr/' + mapr
            v = nsmap[k]
            nsmap = {k: v}
        query = []
        for ns, nsd in nsmap.items():
            t = nsd.short
            p.addString(t + 'ns', ns)
            p.addString(t + 'pk', nsd.pk)
            query.append('(ial.child.ns=:{} AND mvq.name=:{})'.format(
                t + 'ns', t + 'pk'))
        q = """
            SELECT ial.child.id, mvq.value, mv.name, mv.value, ial.child.ns
            FROM ImageAnnotationLink ial
            JOIN ial.child.mapValue AS mv
            JOIN ial.child.mapValue AS mvq
            WHERE ial.parent.id=:id
            AND ial.child.class=MapAnnotation
            AND ({})
            ORDER BY ial.child.id ASC
            """.format(' OR '.join(query))
        #print(q)
        rs = qs.projection(q, p)
        return _annFromQueryResultNs(unwrap(rs))


class Query(graphene.ObjectType):
    image = graphene.Field(Image, id=graphene.Int())
    gene = graphene.List(Gene, key=graphene.String(), value=graphene.String())
    
    def resolve_image(self, info, id):
        p = omero.sys.ParametersI()
        p.addId(id)
        rs = qs.projection("""
            SELECT id, name
            FROM Image
            WHERE id=:id
            """, p)
        if rs:
            return Image(*unwrap(rs[0]))

    def resolve_gene(self, info, key, value):
        p = omero.sys.ParametersI()
        p.page(0, 10)
        p.addString('key', key)
        p.addString('value', value)
        rs = qs.projection("""
            SELECT ann.id, mvq.value, mv.name, mv.value
            FROM MapAnnotation ann
            JOIN ann.mapValue AS mv
            JOIN ann.mapValue AS mvq
            WHERE ann.ns='openmicroscopy.org/mapr/gene'
            AND mvq.name=:key
            AND lower(mvq.value)=:value
            ORDER BY ann.id ASC
            """, p)
        return _annFromQueryResult(unwrap(rs), Gene)


NsDetails = namedtuple('NsDetails', ['short', 'cls', 'pk'])
NSMAP = {
    'openmicroscopy.org/mapr/gene': NsDetails(
        'ge', Gene, 'Gene Symbol'),
    'openmicroscopy.org/mapr/phenotype': NsDetails(
        'ph', Phenotype, 'Phenotype'),
}


def _annFromQueryResult(rs, AnnClass):
    for k, g in groupby(rs, itemgetter(0, 1)):
        id = k[0]
        name = k[1]
        nvs = [NameValue(el[2], el[3]) for el in g]
        ann = AnnClass(id=id, name=name, value=nvs)
        yield ann


def _annFromQueryResultNs(rs):
    for k, g in groupby(rs, itemgetter(0, 1, 4)):
        id = k[0]
        name = k[1]
        ns = k[2]
        nvs = [NameValue(el[2], el[3]) for el in g]
        annClass = NSMAP[ns].cls
        ann = annClass(id=id, name=name, value=nvs)
        yield ann


schema = graphene.Schema(query=Query, types=(Gene, Phenotype))
#print(schema.execute('{gene(key: "Gene Symbol", value: "polo") {id}}').data)
print(schema.execute('{image(id: 1030631) {annotations(mapr: GENE) {name value {name value}}}}').data)

OrderedDict([('image', OrderedDict([('annotations', [OrderedDict([('name', 'polo'), ('value', [OrderedDict([('name', 'Gene Identifier'), ('value', 'CG12306')]), OrderedDict([('name', 'Gene Identifier URL'), ('value', 'http://www.flybase.org/cgi-bin/uniq.html?field=SYN&db=fbgn&context=CG12306')]), OrderedDict([('name', 'Gene Symbol'), ('value', 'polo')])])])])]))])


In [4]:
# TODO
from promise import Promise
from promise.dataloader import DataLoader

class ImageLoader(DataLoader):
    def batch_load_fn(self, keys):
        # Here we return a promise that will result on the
        # corresponding user for each key in keys
        return Promise.resolve([get_user(id=key) for key in keys])


def qlbuild(*args):
    q = ' '.join(qlbuild(*arg) if isinstance(arg, list) else arg
                 for arg in args)
    return '{' + q + '}'


def execprint(*args):
    if len(args) > 1:
        q = qlbuild(*args)
    else:
        q = args[0]
    print('query: ' + q)
    s = schema.execute(q)
    if s.errors:
        print(s.errors)
    else:
        print(json.dumps(s.data, indent=2))

#print(schema.execute('{geneSymbols(symbol: "polo") {id}}').data)
#print(schema.execute('{ geneSymbols(symbol: "polo") {id images {id}}}').data)
execprint('image(id: 1030631)', ['id'])
execprint('image(id: 1030631)', ['id', 'name', 'annotations',
                                 ['__typename', 'id', 'name', 'value',
                                  ['name', 'value']]])

# execprint('image(id: 1030631)', ['id', 'name', 'annotations',
#     ['__typename', 'id', 'name', 'images', ['id', 'name']]])

query: {image(id: 1030631) {id}}
{
  "image": {
    "id": 1030631
  }
}
query: {image(id: 1030631) {id name annotations {__typename id name value {name value}}}}
{
  "image": {
    "id": 1030631, 
    "name": "Primary_062 [Well B11 Field #1]", 
    "annotations": [
      {
        "__typename": "Phenotype", 
        "id": 6672118, 
        "name": "defective cell-cell aggregation", 
        "value": [
          {
            "name": "Phenotype", 
            "value": "defective cell-cell aggregation"
          }, 
          {
            "name": "Phenotype Term Name", 
            "value": "fewer aggregated cells in population phenotype"
          }, 
          {
            "name": "Phenotype Term Accession", 
            "value": "CMPO_0000416"
          }, 
          {
            "name": "Phenotype Term Accession URL", 
            "value": "http://www.ebi.ac.uk/cmpo/CMPO_0000416"
          }
        ]
      }, 
      {
        "__typename": "Gene", 
        "id": 6672872, 
       

In [5]:
execprint('gene(key: "Gene Symbol", value: "polo")', ['id', 'name', 'images',
    ['id', 'name', 'annotations(mapr: PHENOTYPE)', ['id', 'name']]])

query: {gene(key: "Gene Symbol", value: "polo") {id name images {id name annotations(mapr: PHENOTYPE) {id name}}}}
{
  "gene": [
    {
      "id": 6672872, 
      "name": "polo", 
      "images": [
        {
          "id": 1030631, 
          "name": "Primary_062 [Well B11 Field #1]", 
          "annotations": [
            {
              "id": 6672118, 
              "name": "defective cell-cell aggregation"
            }
          ]
        }, 
        {
          "id": 1030633, 
          "name": "Primary_062 [Well B11 Field #3]", 
          "annotations": [
            {
              "id": 6672118, 
              "name": "defective cell-cell aggregation"
            }
          ]
        }, 
        {
          "id": 1030632, 
          "name": "Primary_062 [Well B11 Field #2]", 
          "annotations": [
            {
              "id": 6672118, 
              "name": "defective cell-cell aggregation"
            }
          ]
        }, 
        {
          "id": 931327, 
  